In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import subprocess
import shutil
import time
import pandas as pd
import datetime

import requests
import lxml
from lxml.html import fromstring

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pyperclip

# headers
userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'
headers = {'User-Agent' : userAgent}

# option 
option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

In [ ]:
# 로그인을 위한 함수 구현
def clipboard_input(user_xpath, user_input):
    temp_user_input = pyperclip.paste()  # 사용자 클립보드를 따로 저장

    pyperclip.copy(user_input)
    driver.find_element_by_xpath(user_xpath).click()
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()

    pyperclip.copy(temp_user_input)  # 사용자 클립보드에 저장 된 내용을 다시 가져 옴
    time.sleep(1)

# 파라미터


In [ ]:
ID = 'YOUR_ID'
PASSWORD = 'YOUR_PW'

# 밴드 사이트에 접근

In [ ]:
# chrome.exe로 접근
subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"') # 디버거 크롬 구동

# url 접근
driver = webdriver.Chrome('90/chromedriver.exe', options=option)
url = 'https://band.us/'
driver.get(url)
time.sleep(2)

# 로그인 클릭
driver.find_element_by_xpath('//*[@id="header"]/div/div/a[2]').click()
time.sleep(2)

# 네이버로 로그인 클릭
driver.find_element_by_xpath('//*[@id="login_list"]/li[3]/a/span').click()

# ID, PW 넣고 로그인
login = {
    "id" : ID,
    "pw" : PASSWORD
}

clipboard_input('//*[@id="id"]', login.get("id"))
clipboard_input('//*[@id="pw"]', login.get("pw"))

# 로그인 상태 유지 클릭
driver.find_element_by_xpath('//*[@id="label_login_chk"]').click()

# 로그인하기 클릭
driver.find_element_by_xpath('//*[@id="log.login"]').click()
time.sleep(2)

# 특정 밴드 클릭
driver.find_element_by_xpath('//*[@id="content"]/div/section/div[2]/div/div/ul/li[2]/div/div/a/div[2]/p').click()
time.sleep(2)

# 보기방식 변경
driver.find_element_by_xpath('//*[@id="content"]/section/div[13]/div[1]/div/button[2]').click()
time.sleep(2)

# 정보 크롤링
## 데이터프레임 초기화 주의!!

In [ ]:
# 정보수집
num = 1
df = pd.DataFrame(columns = ['작성자', '일자', '좋아요개수', '댓글개수', '조회수', '내용'])
t1 = time.time()

while True:
    
    # 정보수집 
    
    
    ## 작성자
    try:
        writer = driver.find_element_by_xpath(f'//*[@id="content"]/section/div[14]/div[2]/div/div[{num}]/div/div[2]/div/span/strong').text
    
    ### Stopping Condition: 작성자가 수집이 안되었을때 이는 게시글이 없는 것으로 판단
    except:
        
        try:
            print('== 1차오류 발생 == 스크롤을 추가로 내려봅니다.')
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            writer = driver.find_element_by_xpath(f'//*[@id="content"]/section/div[14]/div[2]/div/div[{num}]/div/div[2]/div/span/strong').text
            
        except:
            print(f'정보가 모두 수집되었거나 오류가 발생하였습니다. == 마지막 게시글의 작성일자: {date}')
            break
        
        
    ## 일자   
    date = driver.find_element_by_xpath(f'//*[@id="content"]/section/div[14]/div[2]/div/div[{num}]/div/div[2]/div/div/a/time').text
    
    if ('시간' in date) | ('분' in date): ### 20시간 전 등의 데이터에 대해서는 오늘 날짜를 기입
        today = datetime.datetime.today()
        date = '%s년 %s월 %s일' % ("{:d}".format(today.year), "{:d}".format(today.month), "{:d}".format(today.day))

        
    ## 좋아요 개수
    like_count = driver.find_element_by_xpath(f'//*[@id="content"]/section/div[14]/div[2]/div/div[{num}]/div/div[4]/div[1]/div[1]/button/div/span[2]').text
    
    
    ## 댓글 개수
    comment_count = driver.find_element_by_xpath(f'//*[@id="content"]/section/div[14]/div[2]/div/div[{num}]/div/div[4]/div[1]/div[2]/a/span[2]').text
    
    
    ## 조회수 개수
    try:
        view_count = driver.find_element_by_xpath(f'//*[@id="content"]/section/div[14]/div[2]/div/div[{num}]/div/div[4]/div[2]/span/span[2]').text
    
    except: ### 조회수가 없으면 해당 element가 활성화 안되어 오류가 발생
        view_count = 0
        
        
    ## 내용
    content = driver.find_element_by_xpath(f'//*[@id="content"]/section/div[14]/div[2]/div/div[{num}]/div/div[3]/div/div/div[1]/p').text

    
    
    
    # 데이터프레임에 적재
    tmp = pd.DataFrame({'작성자':[writer],
                     '일자':[date],
                     '좋아요개수':[like_count],
                     '댓글개수':[comment_count],
                     '조회수':[view_count],
                     '내용':[content]})

    df = df.append(tmp, ignore_index = True)
    
    # scroll down(주기: 5)
    if num % 5 == 0:
        print(f'== {num}개 게시글 수집 완료 == Scroll Down!! == 경과시간: {round(time.time() - t1)}초')
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        
    
    # num 추가
    num += 1

In [ ]:
df.to_excel('../데이터/네이버_밴드_크롤링.xlsx', index=False)